In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [2]:
class NeuronalNetVector(nn.Module):
        def __init__(self):
            super(NeuronalNetVector, self).__init__()
            self.lin1 = nn.Linear(8,30)
            self.lin2 = nn.Linear(30,30)
            self.lin3 = nn.Linear(30,3)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            x = F.softmax(x,dim=0)
            return x

In [3]:
#Die Tore der einzelnen Mannschaften pro Spiel aus den matches in eine Liste schreiben
def inputValuesVector(gameList, df_stats,inputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        input_values= df_stats[df_stats['HomeTeam']==homeTeam]
        input_values= input_values[input_values['AwayTeam']==awayTeam]
        input_values= input_values.reset_index(drop=True)
        input_values = input_values[:-1]
        inputList.append(input_values.loc[:,['FTHG','FTAG']])

In [4]:
#Methode um aus einer Liste eine Tensorliste mit Floatwerten generieren
def dfToTensor(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df.values).view(8)
        tensorList.append(tensor)

In [5]:
def dfToTensor_forTarget(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df).view(3)
        tensorList.append(tensor)

In [6]:
#Outputvalues generieren, bei einem target als Vector
def targetValuesVector(gameList, df_stats,targetList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        target_values= df_stats[df_stats['HomeTeam']==homeTeam]
        target_values= target_values[target_values['AwayTeam']==awayTeam]
        target_values= target_values.reset_index(drop=True)
        target_values = target_values.tail(1)
        target_values = target_values.loc[:,['FTHG','FTAG']]
        homegoals = target_values.loc[:,['FTHG']].iloc[0]['FTHG']
        awaygoals = target_values.loc[:,['FTAG']].iloc[0]['FTAG']
        if homegoals > awaygoals:
            df_target=[1.,0.,0.]
        if homegoals == awaygoals:
            df_target=[0.,1.,0.]
        if homegoals < awaygoals:
            df_target=[0.,0.,1.]
        targetList.append(df_target)

In [10]:
#Pfade Fabian
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv

#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv

In [7]:
def vectorInputTarget(path,inputListe,targetListe):
    topTeams = pd.read_csv(path, sep=';')
    df = pd.DataFrame(topTeams, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
    df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')
    df_names = df_names[['HomeTeam','AwayTeam']]
    gameListe = []
    for row in df_names.itertuples():
        data = [[row.HomeTeam,row.AwayTeam]]
        gameListe.append(pd.DataFrame(data))
    inputValuesVector(gameListe, df,inputListe)
    targetValuesVector(gameListe,df,targetListe)

In [10]:
inputList_0910_1314_Vector = []
targetList_0910_1314_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv',inputList_0910_1314_Vector,targetList_0910_1314_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv',inputList_0910_1314,outputList_0910_1314)

inputList_1011_1415_Vector = []
targetList_1011_1415_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv',inputList_1011_1415_Vector,targetList_1011_1415_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv',inputList_1011_1415,outputList_1011_1415)

inputList_1112_1516_Vector = []
targetList_1112_1516_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv',inputList_1112_1516_Vector,targetList_1112_1516_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv',inputList_1112_1516,outputList_1112_1516)

inputList_1213_1617_Vector = []
targetList_1213_1617_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv',inputList_1213_1617_Vector,targetList_1213_1617_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv',inputList_1213_1617,outputList_1213_1617)

inputList_1314_1718_Vector = []
targetList_1314_1718_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv',inputList_1314_1718_Vector,targetList_1314_1718_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv',inputList_1314_1718,outputList_1314_1718)

inputList_1415_1819_Vector = []
targetList_1415_1819_Vector = []
vectorInputTarget('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv',inputList_1415_1819_Vector,targetList_1415_1819_Vector)
#vectorInputTarget('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv',inputList_1415_1819,outputList_1415_1819)

In [11]:
#Leere Dataframes löschen
for i in range(len(inputList_1415_1819_Vector)):
    if inputList_1415_1819_Vector[i].empty:
        print(i)

11
14
32
46
54
63
85
92
97
109
116


In [12]:
inputList_1415_1819_Vector.pop(11)
inputList_1415_1819_Vector.pop(13)
inputList_1415_1819_Vector.pop(30)
inputList_1415_1819_Vector.pop(43)
inputList_1415_1819_Vector.pop(50)
inputList_1415_1819_Vector.pop(58)
inputList_1415_1819_Vector.pop(79)
inputList_1415_1819_Vector.pop(85)
inputList_1415_1819_Vector.pop(89)
inputList_1415_1819_Vector.pop(100)
inputList_1415_1819_Vector.pop(106)

targetList_1415_1819_Vector.pop(11)
targetList_1415_1819_Vector.pop(13)
targetList_1415_1819_Vector.pop(30)
targetList_1415_1819_Vector.pop(43)
targetList_1415_1819_Vector.pop(50)
targetList_1415_1819_Vector.pop(58)
targetList_1415_1819_Vector.pop(79)
targetList_1415_1819_Vector.pop(85)
targetList_1415_1819_Vector.pop(89)
targetList_1415_1819_Vector.pop(100)
targetList_1415_1819_Vector.pop(106)

print(len(inputList_1415_1819_Vector))
print(len(targetList_1415_1819_Vector))

110
110


In [13]:
train_input_tensor = []
dfToTensor(inputList_0910_1314_Vector,train_input_tensor)
dfToTensor(inputList_1011_1415_Vector,train_input_tensor)
dfToTensor(inputList_1112_1516_Vector,train_input_tensor)
dfToTensor(inputList_1213_1617_Vector,train_input_tensor)
dfToTensor(inputList_1314_1718_Vector,train_input_tensor)

test_input_tensor= []
dfToTensor(inputList_1415_1819_Vector,test_input_tensor)

In [16]:
train_target_tensor = []
dfToTensor_forTarget(outputList_0910_1314_Vector,train_target_tensor)
dfToTensor_forTarget(outputList_1011_1415_Vector,train_target_tensor)
dfToTensor_forTarget(outputList_1112_1516_Vector,train_target_tensor)
dfToTensor_forTarget(outputList_1213_1617_Vector,train_target_tensor)
dfToTensor_forTarget(outputList_1314_1718_Vector,train_target_tensor)

test_target_tensor = []
dfToTensor_forTarget(outputList_1415_1819_Vector,test_target_tensor)

In [17]:
print(len(train_input_tensor))
print(len(train_target_tensor))
print(len(test_input_tensor))
print(len(test_target_tensor))

710
710
110
121


In [18]:
net = NeuronalNetVector()

In [19]:
criterion = nn.L1Loss()

In [20]:
optimizer = torch.optim.SGD(net.parameters(),lr =0.01, momentum = 0.9)

In [22]:
for epoch in range(50):
    running_loss = 0.0
    for i in range(len(train_input_tensor)):
        optimizer.zero_grad()
        outputs = net(train_input_tensor[i])
        loss = criterion(outputs, train_target_tensor[i])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 710 == 709:
            print (epoch + 1 , i+1 , running_loss / 710)
            running_loss = 0.0

1 710 0.3595081256468658
2 710 0.3594969123417487
3 710 0.3594842958726856
4 710 0.3594701569992979
5 710 0.3594547328609287
6 710 0.3594376768229303
7 710 0.3594167127263934
8 710 0.3593927853819629
9 710 0.35936042657636286
10 710 0.35932396240206144
11 710 0.3592747322779161
12 710 0.3592015805649156
13 710 0.3590653903624888
14 710 0.3586773024346366
15 710 0.358138836188244
16 710 0.3577816927407632
17 710 0.3563765088853832
18 710 0.35483922424336883
19 710 0.3510955506036621
20 710 0.35081876648870497
21 710 0.3556501964278378
22 710 0.3514044620739811
23 710 0.34961352713371097
24 710 0.3515123827667167
25 710 0.34948619713871176
26 710 0.34520078682803484
27 710 0.34795949502620055
28 710 0.3525994538110844
29 710 0.348140029881817
30 710 0.3518052396836036
31 710 0.34957180593475623
32 710 0.34893533662411413
33 710 0.3467200936644743
34 710 0.3479127769896931
35 710 0.33736251039645454
36 710 0.33753166598586154
37 710 0.3380655133700434
38 710 0.3477587814623299
39 710 0.34

In [26]:
#Quantitative Evaluierung 
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_input_tensor)) :
        outputs = net(test_input_tensor[i])
        if (torch.argmax(outputs) == torch.argmax(test_target_tensor[i])):
            correct = correct+1
print(correct / len(test_input_tensor))

0.4727272727272727
